In [1]:
import os
import psycopg2
import requests
import json
import datetime
from pytz import timezone
import pandas as pd
import requests

In [7]:
spot = requests.get('https://api.bybit.com/spot/quote/v1/ticker/24hr').json()
df_spot = pd.DataFrame(spot['result'])
df_spot['timestamp'] = pd.to_datetime(df_spot['time'], unit='ms')

In [8]:
df_spot

,time,symbol,bestBidPrice,bestAskPrice,volume,quoteVolume,lastPrice,highPrice,lowPrice,openPrice,timestamp
0,1637243400002,EOSUSDT,4.1732,4.1749,410253.4349,1745363.68015749,4.1737,4.4344,4.1112,4.3562,2021-11-18 13:50:00.002
1,1637243403231,USDCUSDT,1.0002,1.0004,528922.25,529018.03159,1.0004,1.0018,0.9989,0.9997,2021-11-18 13:50:03.231
2,1637243400004,SUSHIUSDT,9.231,9.238,45450.08,432722.211779,9.228,10.143,9.108,9.906,2021-11-18 13:50:00.004
3,1637243400006,XRPBTC,0.00001825,0.00001826,874749.7,16.182438354,0.00001826,0.00001918,0.0000181,0.00001825,2021-11-18 13:50:00.006
4,1637243400005,LUNAUSDT,42.69,42.75,17128.285,733357.23503,42.71,45.18,41.38,42.92,2021-11-18 13:50:00.005
...,...,...,...,...,...,...,...,...,...,...,...
68,1637243400020,GRTUSDT,0.9175,0.9195,359209.37,341958.240985,0.9195,1.0209,0.8978,0.9232,2021-11-18 13:50:00.020
69,1637243400008,PSPUSDT,0.9091,0.9142,233968.52,229502.224979,0.9102,1.2495,0.8696,1.2115,2021-11-18 13:50:00.008
70,1637243400015,CHZUSDT,0.465,0.4657,1888553.65,891803.267588,0.4657,0.4981,0.455,0.49,2021-11-18 13:50:00.015
71,1637243400021,ANKRUSDT,0.1308,0.13099,5049800.8,642193.270254,0.12967,0.13382,0.12242,0.13347,2021-11-18 13:50:00.021


In [9]:
host="rdsnske.c7sczatjnphy.ap-northeast-1.rds.amazonaws.com"
dbname="bot"
user="rdsuser"
password="rds%5678"
insert_base = "insert into bybit_api_spot values('{basetime}','{symbol}','{best_bid_price}','{best_ask_price}','{volume}','{volume_quote}','{last}','{high}','{low}','{open}');"

In [15]:
# データをDBに登録
with psycopg2.connect(host=host, dbname=dbname, user=user, password=password) as conn:
    with conn.cursor() as cur:
        basetime = datetime.datetime.now()
        basetime = basetime.astimezone(timezone('UTC'))
        basetime = datetime.datetime.strftime(basetime, '%Y-%m-%dT%H:%M:00')
        for d in df_spot.itertuples():
            insert_sql = insert_base.format(basetime=basetime
                                            ,symbol=d.symbol
                                            ,best_bid_price=d.bestBidPrice
                                            ,best_ask_price=d.bestAskPrice
                                            ,volume=d.volume
                                            ,volume_quote=d.quoteVolume
                                            ,last=d.lastPrice
                                            ,high=d.highPrice
                                            ,low=d.lowPrice
                                            ,open=d.openPrice)
            cur.execute(insert_sql)
    conn.commit()

In [10]:
df_spot.columns

Index(['time', 'symbol', 'bestBidPrice', 'bestAskPrice', 'volume',
       'quoteVolume', 'lastPrice', 'highPrice', 'lowPrice', 'openPrice',
       'timestamp'],
      dtype='object')

In [14]:
df_spot.to_csv('df_spot.csv')